# Strojové učenie



#### Použité knižnice

In [1]:
import pandas as pd
from IDA_utils import *
from scipy.stats import shapiro
from typing import Literal

#### Predpríprava dát

- Najskôr je nutné načítať zdrojové súbory do Pandas datasetov.
- Ďalej je vhodné rozdeliť si dáta na __trénovací__ a __testovací dataset__ pomocou funcie *split_data()*. Ďalej budeme pracovať iba s trénovacími datasetmi.

In [2]:
PRODUCT_FILE_PATH = "resources/product.csv"
USER_FILE_PATH = "resources/user.csv"
SESSION_FILE_PATH = "resources/session.csv"

prod_df = pd.read_csv(PRODUCT_FILE_PATH, delimiter='\t')
user_df = pd.read_csv(USER_FILE_PATH, delimiter='\t')
sess_df = pd.read_csv(SESSION_FILE_PATH, delimiter='\t')

prod_train_df, prod_test_df = split_data(prod_df, 0.8)
user_train_df, user_test_df = split_data(user_df, 0.8)
sess_train_df, sess_test_df = split_data(sess_df, 0.8)

- Teraz sa môžme vysporiadať z nešpecifikovanými hodnotami a vychýlenými hodnotami. Budeme postupovať rovnako ako v minulej fáze, použijeme funkcie zo súboru __IDA_utils.py__.

In [3]:
process_missing_vals(prod_train_df)
process_missing_vals(user_train_df)
process_missing_vals(sess_train_df)

process_outliers(prod_train_df)
process_outliers(user_train_df)
process_outliers(sess_train_df)


starting the process...
Limit: 120
Column 'location': No NaN values initially.
Column 'product_ean': No NaN values initially.
Column 'store_name': No NaN values initially.
Column 'code': Dropping NaN values...
NaN count before - 19, after - 0
Process done!


starting the process...
Limit: 21
Column 'address': Too many NaN values... Imputing the most common value...
NaN count before - 296, after - 0
Column 'name': No NaN values initially.
Column 'race': Too many NaN values... Imputing the most common value...
NaN count before - 1420, after - 0
Column 'registration': No NaN values initially.
Column 'username': No NaN values initially.
Column 'current_location': Too many NaN values... Imputing the most common value...
NaN count before - 102, after - 0
Column 'sex': Too many NaN values... Imputing the most common value...
NaN count before - 995, after - 0
Column 'birthdate': Too many NaN values... Imputing the most common value...
NaN count before - 902, after - 0
Column 'mail': No NaN va

## Jednoduchý klasifikátor na základe závislostí v dátach

- V tejto časti implementujeme jednoduchý __OneR algoritmus__, ktorý nám s určitou presnosťou určí klasifikuje výsledok skúmaného atribútu pri jednotlivých prediktoroch. Celková funkcionalita je implementovaní pomocou nasledovných funkcií:
    
    a) *frequency_tables()*
    
    b) *one_rule_alg()*

- Prvá z funkcí vracia pre jednotlivé prediktory __tabuľky frekvencií__ vo vzťahu so skúmaných atribútom. V podstate ide o zoskupenie všetkých unikátnych hodnôt a uvedenie ich počtov. Výsledok je vrátený ako mapa, kde kľúče sú názvy prediktorov a hodnoty samotné tabuľky. Druhá funkcia akceptuje ako parameter takúto mapu, kde pre každý prediktor určí pravidlo, či platí hodnota A alebo B a s akou presnosťou to možno určiť.

In [17]:
def frequency_tables(df: pd.DataFrame, target_column: str, predictors: tuple[str]):
    if len(predictors) == 0:
        raise ValueError("At least one predictor needed!")

    result = {}

    for predictor in predictors:
        freq_table = pd.crosstab(df[predictor], df[target_column])
        result[predictor] = freq_table

    return result


def one_rule_alg(freq_tables: Dict[str, pd.DataFrame], metric: Literal['accuracy', 'precision', 'recall'] = 'accuracy'):
    accuracies = {}
    rules = {}

    for table in freq_tables:
        positive = [0, 0]
        negative = [0, 0]

        for index, row in freq_tables[table].iterrows():

            row_vals = row.items()
            row_vals = list(row_vals)      
            if row_vals[0][1] > row_vals[1][1]:
                rules[table] = freq_tables[table].columns[0]
                positive[0] +=  row_vals[0][1]
                positive[1] += row_vals[1][1]
            else:
                rules[table] = freq_tables[table].columns[1]
                negative[0] +=  row_vals[0][1]
                negative[1] += row_vals[1][1]
        
        try:
            if metric == 'accuracy':
                accuracies[table] = ((positive[0] + negative[1])/(positive[0] + positive[1] + negative[0] + negative[1]))
            elif metric == 'precision':
                accuracies[table] = ((positive[0])/(positive[0] + positive[1]))
            elif metric == 'recall':
                accuracies[table] = ((positive[0])/(positive[0] + negative[0]))
        except ZeroDivisionError:
            accuracies[table] = -1

            
    return rules, accuracies, {'used_metric': metric}


- Teraz, keď máme algoritmus implementovaný, môžeme ho ísť otestovať na zvolenom testovacom atribúte a na prediktoroch. V prípade tohto algoritmu sú pravidlá binárne, platí buď hodnota A testoného atribútu alebo B. Preto sme sa rozhodli zvoliť si atribút __ack__ v datasete __sess_train_df__ a budeme skúmať s akou presnosťou by sa stroj rozhodol pre klasifikáciu ďalšieho sedenia na kúpu či nekúpu. Teraz aplikujeme algoritmus na všetky atribúty, o ktorých má zmysel uvažovať ako o možných prediktoroch - vylúčime __ID atribúty__.
- Väčšina atribútov zo spimínaného datasetu je však numerická a nadobúda množstvo rôznych hodnôt. Prediktory by mali mať kategorické hodnoty, preto je nutné rozdeliť ich do niekokoľkých kategórií, a to dokáže nasledovná funkcia:

In [5]:
def num_to_category(df: pd.DataFrame, col: str, num_bins: int = None):

    if not num_bins:
        num_bins = math.floor(df[col].max())        

    df[col] = pd.cut(df[col], bins=num_bins, labels=False)
    df.drop(col, axis=1)

def timestamp_to_category(df: pd.DataFrame, col: str, mode: Literal['day', 'month', 'year'] = 'month'):
    df[col] = pd.to_datetime(df[col])
    # Extract month and create a new column 'month'
    df[col] = df[col].dt.month

- Táto funkcia kategorizuje dataset na základe počtu kategórií v atribúte num_bins. Ak táto hodnota nie je špecifikovaná, je nastavená na maximálnu hodnotu v datasete. Funkcia mení dataset priamo.

In [6]:
predictors = ('browser_name', 'pct_input', 'wild_mouse_duration', 'pct_scroll_move_duration', 'pct_wild_mouse', 'session_start',
              'pct_doubleclick', 'pct_rage_click', 'pct_click', 'page_activity_duration', 'pct_scrandom', 'mouse_move_total_rel_distance',
              'pct_click_product_info', 'scroll_move_total_rel_distance', 'pct_mouse_click')

sess_train_df_copy = sess_train_df.copy()

for predictor in predictors:
    if predictor == 'browser_name':
        continue
    elif predictor == 'session_start':
        timestamp_to_category(sess_train_df_copy, col=predictor)
    else:
        num_to_category(sess_train_df_copy, col=predictor)

### Rozhodnutie na základe jedného atribútu
- Najskôr uskutočníme klasifikáciu našej predikovanej premennej __ack__ len na základe jedného prediktora. Klasifikácie budeme realizovať za použitia rôznych nasledovných metrík:

    a) __accuracy__

    b) __precision__
    
    c) __recall__


 Zvolme si pravdepodobný prediktor __pct_doubleclick__, ktorý by podľa nás mohol dosahovať vysokú hodnotu accurracy s hodnotou ACK=0.

In [14]:
freq_table = frequency_tables(df=sess_train_df_copy, target_column='ack', predictors=('pct_doubleclick',))

print(one_rule_alg(freq_tables=freq_table, metric='accuracy'))
print(one_rule_alg(freq_tables=freq_table, metric='precision'))
print(one_rule_alg(freq_tables=freq_table, metric='recall'))


({'pct_doubleclick': 0.0}, {'pct_doubleclick': 0.689493540642506}, {'used_metric': 'accuracy'})
({'pct_doubleclick': 0.0}, {'pct_doubleclick': 0.7717055971793741}, {'used_metric': 'precision'})
({'pct_doubleclick': 0.0}, {'pct_doubleclick': 0.4434033932641175}, {'used_metric': 'recall'})


- Pre tento prípad sa teda z 44 až 77 percentnou presnosťou rozhodujeme, že sedenie bude zahŕňať kúpu produktu. Teraz zapojíme ostatné prediktory a rozhodneme sa na základe celkovej presnosti.

In [18]:
freq_tables = frequency_tables(df=sess_train_df_copy, target_column='ack', predictors=predictors)

print(one_rule_alg(freq_tables=freq_tables, metric='accuracy'))
print(one_rule_alg(freq_tables=freq_tables, metric='precision'))
print(one_rule_alg(freq_tables=freq_tables, metric='recall'))



({'browser_name': 1.0, 'pct_input': 1.0, 'wild_mouse_duration': 1.0, 'pct_scroll_move_duration': 1.0, 'pct_wild_mouse': 0.0, 'session_start': 1.0, 'pct_doubleclick': 0.0, 'pct_rage_click': 1.0, 'pct_click': 0.0, 'page_activity_duration': 0.0, 'pct_scrandom': 1.0, 'mouse_move_total_rel_distance': 0.0, 'pct_click_product_info': 0.0, 'scroll_move_total_rel_distance': 1.0, 'pct_mouse_click': 1.0}, {'browser_name': 0.5485309248885332, 'pct_input': 0.6076369040813994, 'wild_mouse_duration': 0.658168514919401, 'pct_scroll_move_duration': 0.549559849091117, 'pct_wild_mouse': 0.5553904195724249, 'session_start': 0.5485309248885332, 'pct_doubleclick': 0.689493540642506, 'pct_rage_click': 0.5520749971418772, 'pct_click': 0.6777180747684921, 'page_activity_duration': 0.5486452497999315, 'pct_scrandom': 0.5621355893449183, 'mouse_move_total_rel_distance': 0.6707442551732022, 'pct_click_product_info': 0.5599634160283525, 'scroll_move_total_rel_distance': 0.6367897564879387, 'pct_mouse_click': 0.5485

- Z výsledkov vidno, že testovaná hodnota __ack__ by bola v 9 prípadoch z 15 klasifikovaná ako kúpa. Hodnota istoty sa pohybovala na úrovni od __0 percent__ (*pct_mouse_click* pri metrike recall) až po takmer __77__ percent pri prediktore _pct_doubleclick_ a metóde Precision. 0 percent pri Recall metóde znamená, že neboli namerané žiadne skutočne pozitívne hodnoty. Vidíme, že ten istý atribút pri metóde Precision spôsobil __delenie nulou__ (preto hodnota -1).
- Nakoľko sú počet klasifikácií predikovaného atribútu ako kúpa aj ich presnosť väčšie, na základe viacerých atribútov sa __rozhodujeme sa opäť pre ACK=1__.

### Vyhodnotenie klasifikátora na základe dostupných metrík
- Teraz vyhodnotíme náš algoritmus na základe nasledovných metrík:

    a) __Accurracy__
    b) __Precision__
    c) __Recall__

- Prvú z nich sme implementovali v samotnom algoritme. 

## Trénovanie a vyhodnotenie klasifikátorov strojového učenia

## Optimializácia alias hyperparameter tuning

## Vyhodnotenie vplyvu zvolenej stratégie riešenia na klasifikáciu